In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
import statsmodels.api as sm

font_path = 'C:\\Windows\\Fonts\\gulim.ttc'
font = fm.FontProperties(fname=font_path).get_name()
mpl.rc('font', family=font)

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def divide_time_range(time):
    if time < 600:
        return '00:00~06:00'
    elif time < 1000:
        return '06:00~10:00' # 출근시간
    elif time < 1600:
        return '10:00~17:00'
    elif time < 2000:
        return '17:00~21:00' # 퇴근시간
    else:
        return '21:00~24:00'


# ARIMA 모델 학습 및 예측 함수
def check_best_arima(data):
    for i in range(1,6) : 
        for j in range(1,3) :
            for k in range(1,3) : 
                model = ARIMA(data, order=(i,j,k))  # ARIMA(p,d,q) 모델 설정1
                model_fit = model.fit()
                # 예측 결과 출력
                print(model_fit.summary())  
                
def arima_test(data, IDs) : 
    for 주거_ID in IDs:
        
        # 해당 주거 대여소의 데이터 추출
        주거_data = data[data['시작_대여소_ID'] == 주거_ID]

        # 기준_날짜를 날짜형 데이터로 변환
        주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])

        # 기준_날짜를 인덱스로 설정하고 일별로 집계
        주거_data.set_index('기준_날짜', inplace=True)
        arima_data = 주거_data.resample('D')['전체_건수'].sum()  # 일별 전체_건수 합계 집계

        # ARIMA 모델을 통한 예측
        forecast = check_best_arima(arima_data)

        print(f"대여소 {주거_ID}의 다음 시간대 예측값: {forecast}")


In [29]:
강서_대여소_ID = pd.read_csv('../../Data/따릉이/강서_따릉이_대여소_ID.csv',encoding='utf-8')

강서_데이터1 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_01.csv',encoding='utf-8')
강서_데이터2 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_02.csv',encoding='utf-8')

강서_데이터 = pd.concat([강서_데이터1, 강서_데이터2], ignore_index=True)
print(len(강서_데이터))

top_20_ID = pd.read_csv('../../Data/따릉이/쏠림정도_top20_대여소.csv',encoding='utf-8')

3385613


In [30]:
# 주거와 상업으로 데이터 분할
주거_df = top_20_ID[top_20_ID['최종용도'] == '주거']
상업_df = top_20_ID[top_20_ID['최종용도'] == '상업']

In [31]:
# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

# 강서 데이터에서 대여소_ID가 주거 데이터에 있는 것만 추출
주거_강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(주거_IDs)]
주거_강서_df1 = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(주거_IDs)]

# 강서 데이터에서 대여소_ID가 상업 데이터에 있는 것만 추출
상업_강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(상업_IDs)]
상업_강서_df1 = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(상업_IDs)]

In [32]:
주거_강서_df = 주거_강서_df.sort_values(by='기준_날짜')
주거_강서_df1 = 주거_강서_df.sort_values(by='기준_날짜')
상업_강서_df = 상업_강서_df.sort_values(by='기준_날짜')
상업_강서_df1 = 상업_강서_df.sort_values(by='기준_날짜')

In [33]:
    
주거_강서_df['시간대_구간'] = 주거_강서_df['기준_시간대'].apply(divide_time_range)
주거_강서_df1['시간대_구간'] = 주거_강서_df1['기준_시간대'].apply(divide_time_range)
상업_강서_df['시간대_구간'] = 상업_강서_df['기준_시간대'].apply(divide_time_range)
상업_강서_df1['시간대_구간'] = 상업_강서_df1['기준_시간대'].apply(divide_time_range)



주거_강서_df_출근 = 주거_강서_df[주거_강서_df['시간대_구간'] == '06:00~10:00']
주거_강서_df_퇴근 = 주거_강서_df1[주거_강서_df['시간대_구간'] == '06:00~10:00']
상업_강서_df_출근 = 주거_강서_df[주거_강서_df['시간대_구간'] == '06:00~10:00']
상업_강서_df_퇴근 = 주거_강서_df1[주거_강서_df['시간대_구간'] == '06:00~10:00']
# 전체 파일에서 1차 전처리 (집계_기준 2개 -> 1개로 줄임, 사용량 =0)
주거_강서_df_출근 = 주거_강서_df_출근[(주거_강서_df_출근['전체_이용_분'] != 0) & (주거_강서_df_출근['전체_이용_거리'] != 0)]
주거_강서_df_퇴근 = 주거_강서_df_퇴근[(주거_강서_df_퇴근['전체_이용_분'] != 0) & (주거_강서_df_퇴근['전체_이용_거리'] != 0)]
상업_강서_df_출근 = 상업_강서_df_출근[(상업_강서_df_출근['전체_이용_분'] != 0) & (상업_강서_df_출근['전체_이용_거리'] != 0)]
상업_강서_df_퇴근 = 상업_강서_df_퇴근[(상업_강서_df_퇴근['전체_이용_분'] != 0) & (상업_강서_df_퇴근['전체_이용_거리'] != 0)]


C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\1946588316.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  주거_강서_df_퇴근 = 주거_강서_df1[주거_강서_df['시간대_구간'] == '06:00~10:00']
C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\1946588316.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  상업_강서_df_퇴근 = 주거_강서_df1[주거_강서_df['시간대_구간'] == '06:00~10:00']


In [34]:
arima_test(주거_강서_df1,주거_IDs)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\385166018.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  362
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -1959.959
Date:                Wed, 08 May 2024   AIC                           3925.918
Time:                        13:42:25   BIC                           3937.584
Sample:                    04-03-2023   HQIC                          3930.556
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3455      0.059      5.837      0.000       0.229       0.461
ma.L1         -0.9857      0.013    -74.604      0.000      -1.012      -0.960
sigma2      3019.1475    398.984      7.567      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  362
Model:                 ARIMA(1, 1, 2)   Log Likelihood               -1948.254
Date:                Wed, 08 May 2024   AIC                           3904.508
Time:                        13:42:26   BIC                           3920.063
Sample:                    04-03-2023   HQIC                          3910.692
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0450      0.120     -0.374      0.708      -0.280       0.190
ma.L1         -0.4811      0.102     -4.697      0.000      -0.682      -0.280
ma.L2         -0.4934      0.097     -5.062      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


...

                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  333
Model:                 ARIMA(4, 2, 2)   Log Likelihood               -1846.211
Date:                Wed, 08 May 2024   AIC                           3706.423
Time:                        13:43:23   BIC                           3733.038
Sample:                    05-02-2023   HQIC                          3717.038
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.0579      0.126     -8.402      0.000      -1.305      -0.811
ar.L2         -0.5561      0.107     -5.181      0.000      -0.766      -0.346
ar.L3         -0.4892      0.090     -5.461      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  333
Model:                 ARIMA(5, 1, 2)   Log Likelihood               -1726.044
Date:                Wed, 08 May 2024   AIC                           3468.088
Time:                        13:43:24   BIC                           3498.529
Sample:                    05-02-2023   HQIC                          3480.228
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3507      0.056      6.267      0.000       0.241       0.460
ar.L2         -0.6997      0.058    -11.987      0.000      -0.814      -0.585
ar.L3         -0.2621      0.061     -4.292      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  333
Model:                 ARIMA(5, 2, 2)   Log Likelihood               -1751.544
Date:                Wed, 08 May 2024   AIC                           3519.088
Time:                        13:43:25   BIC                           3549.505
Sample:                    05-02-2023   HQIC                          3531.220
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1105      0.064     -1.715      0.086      -0.237       0.016
ar.L2         -0.4631      0.047     -9.876      0.000      -0.555      -0.371
ar.L3         -0.3668      0.051     -7.190      0.0

In [18]:
grouped_data = 주거_강서_df1.groupby(['기준_날짜', '시작_대여소_ID'])['전체_건수'].sum().reset_index()
print(grouped_data)

           기준_날짜 시작_대여소_ID  전체_건수
0     2023-04-03   ST-1253     27
1     2023-04-03   ST-1352     56
2     2023-04-03   ST-1645     70
3     2023-04-03   ST-2056     61
4     2023-04-03   ST-2557     49
...          ...       ...    ...
1637  2024-03-29   ST-1645     62
1638  2024-03-29   ST-2050     75
1639  2024-03-29   ST-2056     46
1640  2024-03-29   ST-2557     57
1641  2024-03-29   ST-2965     58

[1642 rows x 3 columns]


# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
arima_test(주거_강서_df1,주거_IDs)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\385166018.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\P

                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  362
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -1693.346
Date:                Wed, 08 May 2024   AIC                           3392.693
Time:                        09:56:09   BIC                           3404.359
Sample:                    04-03-2023   HQIC                          3397.331
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3838      0.061      6.304      0.000       0.264       0.503
ma.L1         -0.9896      0.012    -81.155      0.000      -1.014      -0.966
sigma2       688.8429     88.216      7.809      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  362
Model:                 ARIMA(2, 1, 1)   Log Likelihood               -1674.920
Date:                Wed, 08 May 2024   AIC                           3357.839
Time:                        09:56:09   BIC                           3373.395
Sample:                    04-03-2023   HQIC                          3364.024
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.4874      0.072      6.809      0.000       0.347       0.628
ar.L2         -0.3179      0.066     -4.846      0.000      -0.447      -0.189
ma.L1         -0.9733      0.016    -59.535      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


...

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  332
Model:                 ARIMA(5, 1, 2)   Log Likelihood               -1504.697
Date:                Wed, 08 May 2024   AIC                           3025.395
Time:                        09:56:39   BIC                           3055.812
Sample:                    05-03-2023   HQIC                          3037.526
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3673      0.064      5.723      0.000       0.242       0.493
ar.L2         -0.6958      0.057    -12.204      0.000      -0.808      -0.584
ar.L3         -0.2617      0.062     -4.240      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  332
Model:                 ARIMA(5, 2, 2)   Log Likelihood               -1519.623
Date:                Wed, 08 May 2024   AIC                           3055.246
Time:                        09:56:40   BIC                           3085.639
Sample:                    05-03-2023   HQIC                          3067.369
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0902      0.065     -1.384      0.166      -0.218       0.037
ar.L2         -0.4697      0.045    -10.498      0.000      -0.557      -0.382
ar.L3         -0.3884      0.054     -7.241      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------